#### CREATE DATASET

In [3]:
import time
import os

import cv2


font = cv2.FONT_HERSHEY_SIMPLEX
facedata = "classifier/haarcascade_frontalface_default.xml"
cascade = cv2.CascadeClassifier(facedata)
NAME = "deven_pawar"

cam = cv2.VideoCapture(0)

cv2.namedWindow("capture faces")

for i in range(1, 50):
    ret, frame = cam.read()
    if not ret:
        break

    cv2.imshow("capture faces", frame)
    k = cv2.waitKey(1)

    if k%256 == 27:
        print("\nEscape hit, closing...")
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cascade.detectMultiScale(frame)

    for f in faces:
        x, y, w, h = [ v for v in f ]

        sub_face = frame[y:y+h + 50, x:x+w + 50]

        img_name = "a/%s.%s.jpg" % (i, NAME)

        cv2.imwrite(img_name, sub_face)

cam.release()

cv2.destroyAllWindows()

#### Make video

In [ ]:
import os
from collections import Counter

import cv2
import face_recognition
from IPython.display import clear_output
import pickle

cv2.destroyAllWindows()
INPUT_FILE = "1.mp4" 
OUTPUT_FILE = "out_1.avi"
DATASET_FILE_NAME = 'model_encodings.pickle'
RESZIE_WIDTH = 1920
RESIZE_HEIGHT = int(9 / 16 * RESZIE_WIDTH)
FONT = cv2.FONT_HERSHEY_SIMPLEX

data = pickle.loads(open(DATASET_FILE_NAME, "rb").read())
num_names = Counter(data['names'])

vid = cv2.VideoCapture(INPUT_FILE)
check, org = vid.read() 

present_name = list()

if check:
    rows, cols, layer = org.shape

out = cv2.VideoWriter(OUTPUT_FILE, cv2.VideoWriter_fourcc(*'DIVX'), 30, (RESZIE_WIDTH, RESIZE_HEIGHT))

while True:
    j += 1
    check, org = vid.read() 

    if org is None:
        break
    k = cv2.waitKey(1)

    if k % 256 == 27:
        print("\nEscape hit, closing...")
        break
    clear_output(wait=True)
    image = cv2.resize(org, (RESZIE_WIDTH, RESIZE_HEIGHT))

    # rotate image
    M = cv2.getRotationMatrix2D((RESZIE_WIDTH / 2, RESIZE_HEIGHT / 2), -90, 1)
    image = cv2.warpAffine(image, M, (RESZIE_WIDTH, RESIZE_HEIGHT))

    boxes = face_recognition.face_locations(image, model='hop')
    encodings = face_recognition.face_encodings(image, boxes)

    names = []

    for encoding, loc in zip(encodings, boxes):
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        num_prop = 0

        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            name = max(counts, key=counts.get)
            num_prop = counts[name] / num_names[name]

        print("[INFO] detected %s with '%f' accuracy ..." % (name, num_prop))

        names.append(name if num_prop > 0.95 else "Unknown")


    for ((top, right, bottom, left), name) in zip(boxes, names):
        left, top, right, bottom = \
            left - 20,\
            top - 40,\
            right + 20,\
            bottom + 20

        cv2.rectangle(image, (left, top), (right, bottom), (255, 0, 0), 2)
        
        s = cv2.getTextSize(name, FONT, 0.78, 2)
        
        cv2.rectangle(image, (left, top), (left + s[0][0] + 40,  top + s[0][1] + 20), (255, 0, 0), -1)
        name = name.replace("_", " ")
        name = name.title()
       
        cv2.putText(image, name, (left + 20, top + 10 + s[0][1]), FONT, 0.78, (255, 255, 255), 2)

    out.write(image)
    cv2.imshow("recognize faces", image)

vid.release()
out.release()

cv2.destroyAllWindows()

In [6]:
import time
import os

import cv2

font = cv2.FONT_HERSHEY_SIMPLEX
facedata = "classifier/haarcascade_frontalface_default.xml"
cascade = cv2.CascadeClassifier(facedata)
NAME = "michael_douglas"
i = 0

for file in os.listdir('dataset_training/michael_douglas'):
    frame = cv2.imread('dataset_training/michael_douglas/'+file, 0)

    try:
        faces = cascade.detectMultiScale(frame)

        for f in faces:
            x, y, w, h = [ v for v in f ]

            sub_face = frame[y:y+h + 50, x:x+w + 50]

            img_name = "michael_douglas/%s.%s.jpg" % (NAME, i)
            cv2.imwrite(img_name, sub_face)

            i+=1
#         cv2.imwrite("michael_douglas/%s.%s.jpg" % (NAME, i), frame)

#         i+=1
    except:
        print("cant open,", file)
        